In [ ]:
ls

In [ ]:
cd ..

In [ ]:
!source venv/bin/activate

In [ ]:
from bs4 import BeautifulSoup
import re
from datasets import load_dataset
from tqdm.notebook import tqdm
import pandas as pd
import numpy as np

from rag_techniques.simple_rag_giga import SimpleRag
from research.functions import get_natural_questions_sample, get_ms_marco_sample

In [ ]:
from src.ragas import Evaluator
from src.answer_correctness import AnswerCorrectness
ragas = Evaluator()

In [ ]:
model = SimpleRag(collection_name='natural_questions')
df = pd.read_csv('research/data/qa_dataset.csv', index_col=0)

In [ ]:
df

In [ ]:
natural_questions_dataset = {
    "question": [],
    "answer": [],
    "contexts": [],
    "ground_truth": [],
}

for i, (question, answer) in tqdm(df.iterrows(), total=df.shape[0]):
    result = model(question)
    natural_questions_dataset["question"].append(question)
    natural_questions_dataset["answer"].append(result['answer'])
    natural_questions_dataset["contexts"].append(result['context'])
    if pd.isna(answer):
        answer = 'No answer'
    natural_questions_dataset["ground_truth"].append(answer)    

In [ ]:
d = pd.read_csv('research/data/simple_rag_df.csv', index_col=0)

In [ ]:
evaluator = AnswerCorrectness()
res = evaluator.get_correctness(d['question'].tolist(), d['answer'].tolist(), d['ground_truth'].tolist())

In [ ]:
juge = res[1][:200]

In [ ]:
human = d['value'].values.tolist()[:200]

In [ ]:
agreement = sum(e1 == e2 for e1, e2 in zip(juge, human)) / len(human)
print("Доля совпадений:", agreement)

In [ ]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(human, juge))

In [ ]:
import random
import numpy as np

In [ ]:
vals = []
for j in range(5):
    vals.append([])
    random.shuffle(res[1])
    for i in range(1, len(res[1])):
        vals[j].append(np.mean(res[1][:i]))

In [ ]:
from matplotlib import pyplot as plt
for val in vals:
    plt.plot(val)

# RuLang

In [ ]:
model = SimpleRag(collection_name='natural_questions', language='ru')

In [ ]:
natural_questions_dataset = {
    "question": [],
    "answer": [],
    "contexts": [],
    "ground_truth": [],
}

for i, (question, answer) in tqdm(df.iterrows(), total=df.shape[0]):
    result = model(question)
    natural_questions_dataset["question"].append(question)
    natural_questions_dataset["answer"].append(result['answer'])
    natural_questions_dataset["contexts"].append(result['context'])
    if pd.isna(answer):
        answer = 'No answer'
    natural_questions_dataset["ground_truth"].append(answer)    

In [ ]:
d = pd.DataFrame(natural_questions_dataset)
d.to_csv('research/data/simple_ru_rag_df.csv')

In [ ]:
evaluator = AnswerCorrectness()
res = evaluator.get_correctness(natural_questions_dataset['question'], natural_questions_dataset['answer'], natural_questions_dataset['ground_truth'])

In [ ]:
np.mean(res[1])